# Исправление опечаток

In [2]:
import os, re
from string import punctuation
import numpy as np
import json
from collections import Counter
from pprint import pprint
punct = set(punctuation)
from sklearn.metrics import classification_report
import pandas as pd
from IPython.display import display

Возьмем данные с соревнования Dialog Evaluation 2015 по исправлению опечаток. Данные представляют собой набор предложений (правильное - ошибочное). Задача найти слова с ошибками и заменить их на правильный вариант.

Недостатком тут является то, что не всегда можно правильно сопоставить слова правильного предложения и ошибочного (из-за слов с пропущенным или добавленным пробелом). Из статьи авторов корпуса не очень понятно, как они решали эту проблему, поэтому я просто удалил все такие предложения.

In [3]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [4]:
# Посмотрим на пары предложений
print(bad[2])
print(true[2])

Пояним эту мысль.
Поясним эту мысль


In [5]:
# напишем функцию, которая будет сопоставлять слова в правильном и ошибочном варианте
# разобьем предложение по пробелам и удалим пунктуация на границах слов
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [6]:
pprint(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]


Вытащим только неправильные варианты и заодно посчитаем процент ошибок.

In [7]:
mistakes = []
total = 0

for i in range(len(true)):
    
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        if pair[0] != pair[1]:
            mistakes.append(pair)
        
        total += 1

In [8]:
print('Доля ошибок - ', len(mistakes)/total )

Доля ошибок -  0.13034358769476628


In [9]:
mistakes[:5]

[('симпатичнейшее', 'симпатичнейшое'),
 ('апофеозом', 'опофеозом'),
 ('поясним', 'пояним'),
 ('получатся', 'полчатся'),
 ('очень', 'оччччень')]

Обернем в Counter, чтобы сразу увидеть частотные ошибки.

In [10]:
Counter(mistakes).most_common(10)

[(('сегодня', 'седня'), 24),
 (('вообще', 'вобще'), 18),
 (('вообще', 'ваще'), 17),
 (('естественно', 'естесственно'), 17),
 (('хочется', 'хочеться'), 16),
 (('кстати', 'кстате'), 16),
 (('очень', 'ооочень'), 14),
 (('как-то', 'както'), 9),
 (('очень', 'оооочень'), 9),
 (('это', 'ето'), 9)]

Из-за того, что процент ошибок довольно низкий, не очень выгодно будет находить исправление для каждого слова. Нужен какой-то более простой классификатор, который выделит ошибочные слова, чтобы потом только их и редактировать.

Самый простой способ это сделать - составить словарь правильных слов и потом сравнивать с ним. Чтобы не делать этого вручную, можно взять какой-нибудь корпус текстов, прошедщих редактуру. Новостные тексты для этого хорошо подходят.

Я заранее собрал небольшой корпус новостных текстов и немного почистил его удалив отдельную пунктуацию и пунктуацию на границах слов.

In [11]:
corpus = open('corpus_ng.txt', encoding='utf8').read().splitlines()

In [12]:
# нормализация нам тут не нужна так как нужно находить слова в разных формах
print(corpus[1].split()[:10])

['судя', 'по', 'всему', 'русская', 'православная', 'церковь', 'нашла', 'долгожданную', 'национальную', 'идею']


## Задание 1.
Напишите функцию, которая будет предсказывать ошибочные слова на основе корпуса.

In [13]:
vocab = set()

for line in corpus: 
    words = line.lower().split()
    vocab.update(words)

In [14]:
def predict_mistaken(word, vocab):
    '''
    ::input: word, vocabulary
    ::output: 1 or 0
    
    '''
    if word == vocab:
        return 1
    else:
        return 0

In [15]:
# для оценки создайте два списка y_true и y_pred
# пройдитесь по предложениям
# сопоставите слова с помощью функции align_words
# пройдитесь по парам слов и
# если слова одинаковые добавьте в y_true 0 
# если слова разные добавьте в y_true 1
# предскажите ошибочность слова из bad списка 
# добавьте предсказание в список y_pred

y_true = []
y_pred = []

for correct, wrong in zip(true, bad): 
    # сопоставляем слова
    pairs = align_words(correct, wrong)
    for correct_word, wrong_word in pairs: 
        if correct_word == wrong_word: 
            y_true.append(0)
        else: 
            y_true.append(1)
        pred = predict_mistaken(wrong_word, vocab)
        y_pred.append(pred)

In [16]:
len (y_pred) == len(y_true)

True

In [17]:
align_words(true[1], bad[1])

[('апофеозом', 'опофеозом'),
 ('дня', 'дня'),
 ('для', 'для'),
 ('меня', 'меня'),
 ('сегодня', 'сегодня'),
 ('стала', 'стала'),
 ('фраза', 'фраза'),
 ('услышанная', 'услышанная'),
 ('в', 'в'),
 ('новостях', 'новостях')]

In [18]:
# оцените качество с помощью classification_report
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

           0       0.87      1.00      0.93      8707
           1       0.00      0.00      0.00      1305

   micro avg       0.87      0.87      0.87     10012
   macro avg       0.43      0.50      0.47     10012
weighted avg       0.76      0.87      0.81     10012



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### Генерация исправлений

Теперь нужно думать о том, как исправить неправильные слова. Посмотрим как это можно делать на примере известной реализации Питера Норвига.

Основная идея - сделать словарь правильных слов (у нас уже есть), расчитать вероятность каждого слова в корпусе.

In [19]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [20]:
WORDS.most_common(10)

[('в', 67679),
 ('и', 55933),
 ('на', 27860),
 ('не', 21627),
 ('что', 18299),
 ('с', 18224),
 ('по', 13117),
 ('а', 9696),
 ('как', 8958),
 ('к', 8907)]

In [21]:
# фунцкия расчета вероятности слова
N = sum(WORDS.values())
def P(word, N=N): 
    "Вычисляем вероятность слова"
    return WORDS[word] / N

In [22]:
P('апофеоз')

1.8025455548325345e-06

Чтобы найти исправления нужно сгенерировать возможные исправления и выбрать те, которые есть в словаре. Если есть несколько вариантов, то выбрать тот, у котогоро наибольшая вероятность.

In [23]:
def correction(word): 
    "Находим наиболее вероятное похожее слово"
    return max(candidates(word), key=P)

def candidates(word): 
    "Генерируем кандидатов на исправление"
    return (known([word]) or known(edits1(word)) or known(edits2(word)) or [word])

def known(words): 
    "Выбираем слова, которые есть в корпусе"
    return set(w for w in words if w in WORDS)

def edits1(word):
    "Создаем кандидатов, которые отличаются на одну букву"
    letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R]
    transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]
    replaces   = [L + c + R[1:]           for L, R in splits if R for c in letters]
    inserts    = [L + c + R               for L, R in splits for c in letters]
    return set(deletes + transposes + replaces + inserts)

def edits2(word): 
    "Создаем кандидатов, которые отличаются на две буквы"
    return (e2 for e1 in edits1(word) for e2 in edits1(e1))

In [24]:
%%time
correction('слива')

CPU times: user 547 µs, sys: 8 µs, total: 555 µs
Wall time: 563 µs


'слова'

Давайте поподробнее разберем, что происходит в функции edits.

In [25]:
word = 'опефеоз'
splits = [(word[:i], word[i:])    for i in range(len(word) + 1)]

In [26]:
splits[:10]

[('', 'опефеоз'),
 ('о', 'пефеоз'),
 ('оп', 'ефеоз'),
 ('опе', 'феоз'),
 ('опеф', 'еоз'),
 ('опефе', 'оз'),
 ('опефео', 'з'),
 ('опефеоз', '')]

In [27]:
deletes = [L + R[1:] for L, R in splits if R]

In [28]:
deletes[:10]

['пефеоз', 'оефеоз', 'опфеоз', 'опееоз', 'опефоз', 'опефез', 'опефео']

In [29]:
transposes = [L + R[1] + R[0] + R[2:] for L, R in splits if len(R)>1]

In [30]:
transposes[:10]

['поефеоз', 'оепфеоз', 'опфееоз', 'опеефоз', 'опефоез', 'опефезо']

In [31]:
letters    = 'йцукенгшщзхъфывапролджэячсмитьбюё'
replaces = [L + c + R[1:] for L, R in splits if R for c in letters]

In [32]:
len(replaces)

231

In [33]:
inserts = [L + c + R for L, R in splits for c in letters]

In [34]:
inserts[:10]

['йопефеоз',
 'цопефеоз',
 'уопефеоз',
 'копефеоз',
 'еопефеоз',
 'нопефеоз',
 'гопефеоз',
 'шопефеоз',
 'щопефеоз',
 'зопефеоз']

Для оценки используем просто долю правильных исправлений.

In [35]:
# До этого мы уже считали долю ошибок во всех предложениях.
# Поэтому если ничего не менять то доля правильных исправлений уже будет 100 - 13 = 87 %.
# Наш подход соответственно должен показывать лучший результат 
correct = 0
total = 0

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i])
    
    for pair in word_pairs:
        
        predicted = correction(pair[1])
        if predicted == pair[0]:
            correct += 1
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.7333333333333333
10
0.8384615384615385
20
0.8441558441558441
30
0.8434343434343434
40
0.847953216374269
50
0.8518518518518519
60
0.8458333333333333
70
0.8502475247524752
80
0.8501594048884166
90
0.8526522593320236
100
0.8512544802867383
110
0.8547439126784215
120
0.8546017014694509
130
0.8534798534798534
140
0.8523581681476419
150
0.8580481622306717
160
0.8573113207547169
170
0.8568220101066817
180
0.8541666666666666
190
0.8552104208416834
200
0.8581048581048581
210
0.8593466424682396
220
0.8598971722365039
230
0.859958932238193
240
0.8639350599149594
250
0.8619170033051781
260
0.8628209637706648
270
0.8603786342123056
280
0.8606001936108422
290
0.8584758942457231
300
0.859375
310
0.8601173020527859
320
0.8605046781967678
330
0.8591549295774648
340
0.8582974137931034
350
0.8576288929599581
360
0.8575772934617334
370
0.8583478045150087
380
0.8595898673100121
390
0.8602558029369967
400
0.8619809788912085
410
0.8613615870153292
420
0.8614124418862077
430
0.8616013775290573
440
0.86276

In [36]:
print(correct/total)

0.8600679184978026


In [37]:
%%time
correction('нав')

CPU times: user 205 µs, sys: 0 ns, total: 205 µs
Wall time: 211 µs


'на'

In [38]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 3.42 s, sys: 30.9 ms, total: 3.45 s
Wall time: 3.5 s


'насмехатьсяаававттававаываываы'

Посмотрим какие исправления выбираются для самых частотных опечаток.

In [39]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)]

[('сегодня', 'седня', 'сеня'),
 ('вообще', 'вобще', 'вообще'),
 ('вообще', 'ваще', 'ваще'),
 ('естественно', 'естесственно', 'естественно'),
 ('хочется', 'хочеться', 'хочется'),
 ('кстати', 'кстате', 'кстати'),
 ('очень', 'ооочень', 'очень'),
 ('как-то', 'както', 'какао'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'ето')]

### Метрики близости слов.

Вместо того, чтобы генерировать все варианты, можно искать похожие слова в словаре. Для этого нужно задать метрику похожести. Для исправления опечаток часто используются расстояния редактирования (количество редактирований, которые нужно сделать в строке a, чтобы прийти к b.

In [ ]:
# в питоне есть библиотеке для нахождения близких строк

In [40]:
from difflib import get_close_matches

In [41]:
%%time
get_close_matches('опофеоз', WORDS.keys(), n=1)

CPU times: user 875 ms, sys: 4.86 ms, total: 880 ms
Wall time: 881 ms


['апофеоз']

Работает тоже не очень быстро.

Недавно вышла библиотека, в которой реализованы многие методы нахождения расстояний.

In [ ]:
!pip install textdistance

In [42]:
import textdistance

In [43]:
def get_closest_match_with_metric(text, lookup, metric=textdistance.levenshtein):
    similarities = Counter()
    for word in lookup:
        similarities[word] = metric.normalized_similarity(text, word) 
    
    return similarities.most_common(1)[0]

In [44]:
%%time
get_closest_match_with_metric('опофиоз', WORDS, textdistance.hamming)

CPU times: user 3.78 s, sys: 24.6 ms, total: 3.8 s
Wall time: 3.82 s


('апофеоз', 0.7142857142857143)

In [45]:
%%time
get_closest_match_with_metric('апофиоз', WORDS, textdistance.levenshtein)

CPU times: user 46.5 s, sys: 481 ms, total: 47 s
Wall time: 46.7 s


('апофеоз', 0.8571428571428572)

Ждать так долго мы не можем, поэтому попробуем что-то побыстрее.

Многие вещи, которые медленно решаются в питоне, можно оптимизировать с помощью матричных и векторных операций.

Сделаем поиск похожих по векторам символов, из которых состоит слово.

In [46]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

In [47]:
corpus = [sent.split() for sent in open('corpus_ng.txt', encoding='utf8').read().splitlines()]
WORDS = Counter()
for sent in corpus:
    WORDS.update(sent)

In [48]:
vocab = list(WORDS.keys())
id2word = {i:word for i, word in enumerate(vocab)}

vec = TfidfVectorizer(analyzer='char', ngram_range=(1,1))
X = vec.fit_transform(vocab)

In [49]:
def get_closest_match_vec(text, X, vec, TOPN=3):
    v = vec.transform([text])
    similarities = cosine_distances(v, X)
    topn = similarities.argsort()[0][:TOPN]
    
    return [id2word[top] for top in topn]

In [50]:
%%time
get_closest_match_vec('опофеоз', X, vec)

CPU times: user 43.5 ms, sys: 11.3 ms, total: 54.8 ms
Wall time: 54.5 ms


['апофеоз', 'апофеозом', 'апофеоза']

## Задание 2. 


Напишите функцию, которая принимает слово и находит ближайшее к нему в словаре (сгенерируйте несколько кандидатов с помощью get_closest_match_vec, а затем посчитайте метрики близости до каждого слова и выведете самое близкое). 

In [51]:
def get_closest_hybrid_match(text, X, vec, metric=textdistance.levenshtein):
    candidates = get_closest_match_vec(text, X, vec, 5)
    closest = get_closest_match_with_metric(text,  candidates, metric)[0]
    return closest

In [52]:
get_closest_hybrid_match('алкогнль', X, vec)

'алкоголь'

In [53]:
# оцените качество также как и раньше (если будет долго работать возьмите кусок данных)
correct = 0
total = 0
incorrect = {'correct word':[], 'wrong word':[], 'predicted word':[]}

for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i]) 
    
    for pair in word_pairs:
        
        predicted = get_closest_hybrid_match(pair[1], X, vec)
        if predicted == pair[0]:
            correct += 1
        else: 
            incorrect['correct word'].append(pair[0])
            incorrect['wrong word'].append(pair[1])
            incorrect['predicted word'].append(predicted)
        total += 1
    if not i % 10:
        print(i)
        print(correct/total)

0
0.5333333333333333
10
0.8076923076923077
20
0.8095238095238095
30
0.8106060606060606
40
0.8128654970760234
50
0.8164251207729468
60
0.8083333333333333
70
0.8118811881188119
80
0.8161530286928799
90
0.8172888015717092
100
0.8127240143369175
110
0.8161209068010076
120
0.8190255220417634
130
0.8175824175824176
140
0.8140806561859193
150
0.8225602027883396
160
0.8213443396226415
170
0.8220101066816395
180
0.8194444444444444
190
0.8211422845691383
200
0.823953823953824
210
0.8262250453720508
220
0.8269065981148244
230
0.8279260780287474
240
0.8326246617703904
250
0.82959970620639
260
0.8301090397467464
270
0.828262339418526
280
0.829945143594708
290
0.8276827371695179
300
0.8293269230769231
310
0.8296187683284457
320
0.8290331726679898
330
0.8282242474454571
340
0.8273168103448276
350
0.8267469248887725
360
0.8269133299543842
370
0.8280823616968495
380
0.8287092882991556
390
0.829227854097584
400
0.8302018093249826
410
0.8307033363390441
420
0.8304184193048484
430
0.8306069737408524
440
0

In [54]:
# смотрим на ошибки
df = pd.DataFrame(incorrect)
display(df)

,correct word,wrong word,predicted word
0,симпатичнейшее,симпатичнейшое,пластичнейшими
1,шпионское,шпионское,шпионские
2,гламурный,гламурный,лагерный
3,бонда,бонда,банд
4,superheadz,superheadz,sueddeutsche
5,clap,clap,place
6,camera,camera,america
7,получатся,полчатся,ополчатся
8,язычки,язычки,язычка
9,очень,оччччень,чечни


В рассмотренных методах при выборе исправления никак не использовался контекст. Про то как это можно сделать, можно почитать вот тут - https://habr.com/ru/post/346618/